In [ ]:
from Repsycle.interact_utils import InteractAPI
import torchvision
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch import nn, optim
from torchvision import models, transforms
from torch.utils.data import Dataset
from torch.optim import lr_scheduler
import random
import numpy as np
from io import BytesIO
import matplotlib.pyplot as plt
import base64
import PIL
from PIL import Image
import json
import os
import cv2
from sklearn.model_selection import StratifiedShuffleSplit
import time
import copy
from tqdm import tqdm
import pandas as pd
import torch
import random
import numpy as np
import yaml
import shutil
from IPython.display import JSON

# Set variables

In [ ]:
# Set interact parameters 
username = None
assert username is not None
password = None
assert password is not None
api_root = "https://interact-api.psycle.io/api/v1"
project_id = None  # Project name
assert password is not None
analysis_id = None  # Analysis name
assert analysis_id is not None

filters = {'annotations__isnull': False, 'annotations__analysis': analysis_id}

# Set file structure 
project_path = f'{os.getcwd()}'

image_folder = f"{project_path}/images"
datas_folder = f"{project_path}/annotations"
weights_folder = f'{project_path}/weights'
augmented_image_folder = f'{project_path}/augmented_images'

datas_path = f'{datas_folder}/datas.json'
train_annotations_path = f'{datas_folder}/train_annotations.json'
val_annotations_path = f'{datas_folder}/val_annotations.json'

train_annotations_yolact_path = f'{datas_folder}/train_annotations_yolact.json'
val_annotations_yolact_path = f'{datas_folder}/val_annotations_yolact.json'


image_extension = 'png'

seed = 42

# Create file structure
if not os.path.exists(image_folder):
    os.mkdir(image_folder)
if not os.path.exists(datas_folder):
    os.mkdir(datas_folder)
if not os.path.exists(weights_folder):
    os.mkdir(weights_folder)
if not os.path.exists(augmented_image_folder):
    os.mkdir(augmented_image_folder)

In [ ]:
def save_json(path,file_to_save):
    with open(path,'w') as f:
        json.dump(file_to_save, f)
        
def open_json(path):
    with open(path, 'r') as f:
        file = json.load(f)
    return file

def get_image_path(data_id: str):
    return f'{image_folder}/{data_id}'

# Get API

In [ ]:
interactAPI = InteractAPI(username, password, api_root)

### Download all annotations

In [ ]:
datas = interactAPI.get_datas(project_id, **filters)
save_json(datas_path, datas)

In [ ]:
datas = open_json(datas_path)
print(f'Number of images: {len(datas)}')

### Set labels

In [ ]:
label_to_int = {}
for idx, label in enumerate(datas[0]['annotations'][0]['analysis']['labels']):
    label_to_int[label] = idx
label_to_int['background'] = idx+1
int_to_label = {v: k for k, v in label_to_int.items()}

print(f'label_to_int: {label_to_int}')
print(f'int_to_label: {int_to_label}')

### Download all images 

In [ ]:
data_ids = [data['id'] for data in datas]
interactAPI.download_images(project_id, data_ids, images_folder_path=image_folder, verbose=True)

In [ ]:
# datas = open_json(datas_path)

# for data in tqdm(datas):
#     data_id = data['id']
#     image_path = get_image_path(data_id)
#     if not os.path.exists(image_path):
#         image = interactAPI.get_image(project_id, data_id, base64=True)
#         image = base64.b64decode(image)
#         image = Image.open(BytesIO(image))
#         image.save(image_path)

### Filter datas

In [ ]:
datas = open_json(datas_path)
filtered_datas = {}

for data in datas:
    data_id = data['id']
    
    labels = []
    coordinates = []
    viewed = data['viewed']
    annotations = data['annotations']
    
    if len(annotations) > 0:
        for idx in range(len(annotations)):
            labels.append(annotations[idx]['label'])
            coordinates.append(annotations[idx]['coordinates'])
            
    if len(labels) == 0:
        labels = ['background']
    
    if data['viewed']:        
        filtered_datas[data_id] = {
            'label': labels,
            'coordinates': coordinates
        }

### Visualize some image

In [ ]:
width, height = 4, 4
fig, axs = plt.subplots(width,height, figsize=(15, 15))
fig.subplots_adjust(hspace = .5, wspace=.001)
axs = axs.ravel()

keys = list(filtered_datas.keys())
annotations = list(filtered_datas.values())
idxs = np.random.randint(0, len(keys), width * height)

for plot_idx, i in enumerate(idxs):
    data_id = keys[i]
    img = cv2.imread(get_image_path(data_id))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h,w,_ = img.shape
    size = (w,h)
    coordinates = annotations[i]['coordinates']
    classes = annotations[i]['label']
    
    for coord in coordinates:
        # Format coords for cv2 contours
        contours = np.zeros(((len(coord) // 2), 2))
        for i in range(0, len(coord), 2):
            contours[i//2] = [coord[i], coord[i+1]]
        contours = contours.astype(int)
        cv2.polylines(img, pts =[contours], isClosed=True, color=(255, 0, 0), thickness=5)

    axs[plot_idx].imshow(img)
    axs[plot_idx].set_title(label)

### Labels distribution

In [ ]:
labels = []
for key, value in filtered_datas.items():
    for label in value['label']:
        labels.append(label)

plt.plot(figsize=(20,20))
plt.hist(labels)
plt.show()

### Split train and test

In [ ]:
y = []
for key, value in filtered_datas.items():
    y.append(value['label'][0])

image_classes = [value['label'] for key, value in filtered_datas.items()]
image_coordinates = [value['coordinates'] for key, value in filtered_datas.items()]
image_ids = list(filtered_datas.keys())

X = np.asarray(image_ids)
y = np.asarray(y)

val_size = int(0.2*len(X))

sss = StratifiedShuffleSplit(n_splits=1, test_size=val_size, random_state=seed)
split = sss.split(X, y)
train_index, test_index = next(split)


train_annotations = {}

for idx in train_index:
    train_annotations[image_ids[idx]] = {
        'label':image_classes[idx], 
        'coordinates':image_coordinates[idx]
    }

val_annotations = {}
for idx in test_index:
    val_annotations[image_ids[idx]] = {
        'label':image_classes[idx], 
        'coordinates':image_coordinates[idx]
    }

save_json(train_annotations_path, train_annotations)
save_json(val_annotations_path, val_annotations)

print(f'Number of annotations for training: {len(train_annotations)}')
print(f'Number of annotations for validation: {len(val_annotations)}')

In [ ]:
labels = []
for key, value in train_annotations.items():
    for label in value['label']:
        labels.append(label)

plt.plot(figsize=(20,20))
plt.hist(labels)
plt.show()
print('Training distribution')

In [ ]:
labels = []
for key, value in val_annotations.items():
    for label in value['label']:
        labels.append(label)

plt.plot(figsize=(20,20))
plt.hist(labels)
plt.show()
print('Validation distribution')

# Load on yolact

In [ ]:
!git clone git@github.com:PsycleResearch/yolact.git

### Get image shape

In [ ]:
data_id = list(val_annotations.keys())[0]
image_path = get_image_path(data_id)
shape = cv2.imread(image_path).shape

In [ ]:
labels = list(label_to_int.keys())
labels.remove('background')

categories = [{
        "supercategory": None, 
        "id": i+1,
        "name": label
    } for i, label in enumerate(labels)]

labels_categories_mapping = {}
for category in categories:
    labels_categories_mapping[category['name']] = category['id']


for annotations_set in [(train_annotations, train_annotations_yolact_path), (val_annotations, val_annotations_yolact_path)]:
    yolact_annotations = []
    yolact_images = []
    nb_annotations = 0

    for i, data_id in tqdm(enumerate(annotations_set[0].keys())):
        image_path = get_image_path(data_id)
        yolact_images.append({
            "license": 0, 
            "url": None, 
            "file_name": image_path, 
            "height": shape[0], 
            "width": shape[1], 
            "date_captured": None, 
            "id": i
        })

        annotations = annotations_set[0][data_id]
        for j in range(len(annotations['label'])):
            label = annotations['label'][j]
            coordinates = annotations['coordinates'][j]
            xs = []
            ys = []
            for k in range(0, len(coordinates), 2):
                xs.append(coordinates[k])
                ys.append(coordinates[k+1])
            xleft = float(np.round(min(xs)))
            xright = float(np.round(max(xs)))
            ytop = float(np.round(min(ys)))
            ybottom = float(np.round(max(ys)))

            coords = []
            for k in range(0, len(coordinates), 2):
                coords.append([coordinates[k], coordinates[k+1]])
            coords = np.array(coords, dtype=np.int64)

            yolact_annotations.append({
                'id': nb_annotations,
                'image_id': i,
                'category_id': labels_categories_mapping[label],
                'segmentation': [coordinates],
                'bbox': [xleft, ytop, xright-xleft, ybottom-ytop],
                'area': float(cv2.contourArea(coords)),
                'iscrowd': 0

            })
            nb_annotations += 1
            
    annotations_yolact = {
        "info": {
            "description": None, 
            "url": None, 
            "version": None, 
            "year": 2021, 
            "contributor": None, 
            "date_created": "2021-08-30 13:53:30.739511"
        }, 
        "licenses": [
            {
                "url": None, 
                "id": 0, 
                "name": None
            }
        ], 
        "images": yolact_images, 
        "type": "instances", 
        "annotations": yolact_annotations,
        'categories': categories
    }

    save_json(annotations_set[1], annotations_yolact)
    

### Yolact file modifications
In data/config.py, change "my_custom_dataset" dict l.104

Number of epochs can be changed in "yolact_base_config", l.519

python yolact/train.py --batch_size=4 --resume weights/yolact_base_198_200000.pth


# Post weights

In [ ]:
preprocessing = {}
augmentations = {}
order = []
hyperparameters = {
    'seed':seed
}

with open(train_annotations_path) as f:
    train_annotations = json.load(f)
training_set = list(train_annotations.keys())
with open(val_annotations_path) as f:
    val_annotations = json.load(f)
val_set = list(val_annotations.keys())

In [ ]:
post_dict = {
    'name':'',   # TODO
    'analysis_id': '1d48c0e4-e46e-4260-8a2d-809d425fb5da',   # ascodero - cuivre
    'hyperparameters': hyperparameters, 
    'labels_output_mapping': label_to_int, 
    'train_set': training_set, 
    'validation_set':val_set,
    'metadata':{
        'model':'YOLACT', 
    }
}

weights_path = None  # TODO
assert weights_path is not None
weights_choice = ""  # TODO
assert weights_choice is not None

In [ ]:
training_id = interactAPI.post_training(post_dict)
print(training_id)
if interactAPI.post_weights(weights_path, training_id['id']) == 204:
    print('Sucess')
interactAPI.put_training_weights_choice(weights_choice, training_id['id'])